In [218]:
import neurokit2 as nk
import numpy as np
import pandas as pd
import tqdm
import matplotlib.pyplot as plt
from biosppy.signals import ecg
from scipy import fftpack
from sklearn.preprocessing import normalize

In [2]:
X_train = pd.read_csv("X_train.csv").drop(columns=["id"])

In [215]:
info = ecg.ecg(pat, sampling_rate =300, show=False)
np.mean(info[4]

array([[ 5.42564934e+01,  5.06679232e+01,  4.75045855e+01, ...,
         3.38392757e+01,  2.58068737e+01,  1.52189267e+01],
       [-2.99455530e+01, -2.42649288e+01, -1.95512517e+01, ...,
        -3.69083834e+01, -2.13244977e+01, -3.76182104e+00],
       [-3.73806647e+00, -1.89161884e+00, -1.40955712e-01, ...,
         4.90869044e-01,  7.05302431e+00,  1.43398545e+01],
       ...,
       [-2.00746191e+01, -2.06669274e+01, -2.09863976e+01, ...,
        -2.02362824e+01, -1.41537120e+01, -6.69066205e+00],
       [-2.25612959e+01, -2.07742808e+01, -1.69330635e+01, ...,
         5.28747507e+00,  5.10407815e+00,  4.80550491e+00],
       [-1.04779096e+01, -7.53463269e+00, -4.99859366e+00, ...,
        -4.64357806e-02, -2.83947056e+00, -5.74631297e+00]])

In [324]:
def feature_extraction(pat):
    #extract peaks and other useful information usign biosppy and neurokit2
    info = ecg.ecg(pat, sampling_rate =300, show=False)
    _, rpeaks = nk.ecg_peaks(info['filtered'], sampling_rate=300, correct_artifacts=False)
    peaks = np.copy(rpeaks['ECG_R_Peaks'])[0:-1] # las value led to index error
    signal, wave = nk.ecg_delineate(info['filtered'], peaks, sampling_rate=300)
    epochs = nk.ecg_segment(signal, rpeaks=rpeaks["ECG_R_Peaks"], sampling_rate=300, show=False)
    hrv_indices = nk.hrv_time(rpeaks["ECG_R_Peaks"], sampling_rate=300, show=False)
    
    # transoform wave item values to numpy arrays
    for key,value in wave.items():
        value= np.array(value)
        wave[str(key)] = np.array(value[~np.isnan(value)],dtype=int)
        
    # calculate number of indices between rpeaks
    rPeakIntervals = []
    for a in range(0, len(rpeaks["ECG_R_Peaks"])-1):
        rPeakIntervals.append(rpeaks["ECG_R_Peaks"][a+1]-rpeaks["ECG_R_Peaks"][a])
    rPeakIntervals = np.array(rPeakIntervals)
    
    rPeakIntervals = []
    for a in range(0, len(rpeaks["ECG_R_Peaks"])-1):
            rPeakIntervals.append(rpeaks["ECG_R_Peaks"][a+1]-rpeaks["ECG_R_Peaks"][a])
    rPeakIntervals = np.array(rPeakIntervals)
        
    qrsInterval = []
    for b in range(0,len(wave["ECG_S_Peaks"])):
        qrsInterval.append(wave["ECG_S_Peaks"][b]-rpeaks["ECG_R_Peaks"][b])
    qrsInterval = np.array(qrsInterval)
    
    qrInterval = []
    for e in range(0,len(wave["ECG_Q_Peaks"])):
        qrInterval.append(rpeaks["ECG_R_Peaks"][e]-wave["ECG_Q_Peaks"][e])
    qrInterval = np.array(qrInterval)
    
    rsInterval = []
    for f in range(0,len(wave["ECG_S_Peaks"])):
        rsInterval.append(wave["ECG_S_Peaks"][b]-rpeaks["ECG_R_Peaks"][b])
    rsInterval = np.array(rsInterval)
    
    pInterval = []
    for c in range(1,len(wave["ECG_P_Peaks"])):
        pInterval.append(wave["ECG_P_Peaks"][c]-wave["ECG_P_Peaks"][c-1])
    pInterval = np.array(pInterval)
    
    prInterval = []
    for d in range(0,len(wave["ECG_P_Peaks"])):
        prInterval.append(rpeaks["ECG_R_Peaks"][d]-wave["ECG_P_Peaks"][d])
    prInterval = np.array(prInterval)
    
    #quality meassure 
    quality = nk.ecg_quality(info['filtered'], sampling_rate=300)
    #something like heart rate
    ecg_rate = nk.signal_rate(_, sampling_rate=300, desired_length=len(_))
    #not realy heart rate
    heartRate = nk.ecg_rsp(ecg_rate, sampling_rate=300)
    
    HRV_MeanNN = hrv_indices['HRV_MeanNN'][0]
    HRV_SDNN = hrv_indices['HRV_SDNN'][0]
    HRV_RMSSD = hrv_indices['HRV_RMSSD'][0]
    HRV_SDSD = hrv_indices['HRV_SDSD'][0]
    
    #means for all intervals
    rPeakIntervalMean = np.nanmean(rPeakIntervals)
    qrsIntervalMean = np.nanmean(qrsInterval)
    qrIntervalMean = np.nanmean(qrInterval)
    rsIntervalMean = np.nanmean(rsInterval)
    pIntervalMean = np.nanmean(pInterval)
    prIntervalMean = np.nanmean(prInterval)
    
    #std deviation for all intervals
    rPeakIntervalStd = np.nanstd(rPeakIntervals)
    qrsIntervalStd = np.nanstd(qrsInterval)
    qrIntervalStd = np.nanstd(qrInterval)
    rsIntervalStd = np.nanstd(rsInterval)
    pIntervalStd = np.nanstd(pInterval)
    prIntervalStd = np.nanstd(prInterval)

    #Max for all intervals
    rPeakIntervalMax = np.nanmax(rPeakIntervals)
    qrsIntervalMax = np.nanmax(qrsInterval)
    qrIntervalMax = np.nanmax(qrInterval)
    rsIntervalMax = np.nanmax(rsInterval)
    pIntervalMax = np.nanmax(pInterval)
    prIntervalMax = np.nanmax(prInterval)
    
    #Min for all intervals
    rPeakIntervalMin = np.nanmin(rPeakIntervals)
    qrsIntervalMin = np.nanmin(qrsInterval)
    qrIntervalMin = np.nanmin(qrInterval)
    rsIntervalMin = np.nanmin(rsInterval)
    pIntervalMin = np.nanmin(pInterval)
    prIntervalMin = np.nanmin(prInterval)
    
    #mean of heartbeat hight
    pPeaksMean = np.nanmean([info['filtered'][l] for l in wave["ECG_P_Peaks"]])
    tPeaksMean = np.nanmean([info['filtered'][l] for l in wave["ECG_T_Peaks"]])
    rPeaksMean = np.nanmean([info['filtered'][l] for l in rpeaks["ECG_R_Peaks"]])
    qPeaksMean = np.nanmean([info['filtered'][l] for l in wave["ECG_Q_Peaks"]])
    sPeaksMean = np.nanmean([info['filtered'][l] for l in wave["ECG_S_Peaks"]])
    sPeaksMin = np.nanmin([info['filtered'][l] for l in wave["ECG_S_Peaks"]])
    tOffsetMean = np.nanmean([info['filtered'][l] for l in wave["ECG_T_Offsets"]])
    pOnsetMean = np.nanmean([info['filtered'][l] for l in wave["ECG_P_Onsets"]])
    
    #std of hearbeat hight
    pPeaksStd = np.nanstd([info['filtered'][l] for l in wave["ECG_P_Peaks"]])
    tPeaksStd = np.nanstd([info['filtered'][l] for l in wave["ECG_T_Peaks"]])
    rPeaksStd = np.nanstd([info['filtered'][l] for l in rpeaks["ECG_R_Peaks"]])
    qPeaksStd = np.nanstd([info['filtered'][l] for l in wave["ECG_Q_Peaks"]])
    sPeaksStd = np.nanstd([info['filtered'][l] for l in wave["ECG_S_Peaks"]])
    tOffsetStd = np.nanstd([info['filtered'][l] for l in wave["ECG_T_Offsets"]])
    pOnsetStd = np.nanstd([info['filtered'][l] for l in wave["ECG_P_Onsets"]])
    
    #summed variance over the overlayed heartbeats
    summed_variance_heartbeat = np.sum(np.std(normalize(info[4]), axis=0))
    
    
    heartRateMean = np.nanmean(heartRate)
    qualityMean = np.nanmean(quality)
    ecgRateMean = np.nanmean(ecg_rate)
    
    heartRateStd = np.nanstd(heartRate)
    qualityStd = np.nanstd(quality)
    ecgRateStd = np.nanstd(ecg_rate)
    
    
    
    
    features = []
    
    features.append([HRV_MeanNN,
                         HRV_SDNN,
                         HRV_RMSSD,
                         HRV_SDSD,
                         rPeakIntervalMean,
                         qrsIntervalMean,
                         qrIntervalMean,
                         rsIntervalMean,
                         pIntervalMean,
                         prIntervalMean,
                         pPeaksMean, 
                         tPeaksMean,
                         rPeaksMean, 
                         qPeaksMean,
                         sPeaksMean,
                         sPeaksMin,
                         tOffsetMean,
                         pOnsetMean,
                         heartRateMean,
                         qualityMean,
                         rPeakIntervalStd,
                         qrsIntervalStd,
                         qrIntervalStd,
                         rsIntervalStd,
                         pIntervalStd,
                         prIntervalStd,
                         pPeaksStd,
                         tPeaksStd,
                         rPeaksStd,
                         qPeaksStd,
                         sPeaksStd,
                         tOffsetStd,
                         heartRateStd,
                         qualityStd,
                         rPeakIntervalMax,
                         qrsIntervalMax,
                         qrIntervalMax,
                         rsIntervalMax,
                         pIntervalMax,
                         prIntervalMax,
                         rPeakIntervalMin,
                         qrsIntervalMin,
                         qrIntervalMin,
                         rsIntervalMin,
                         pIntervalMin,
                         prIntervalMin,
                         summed_variance_heartbeat,
                         np.log(.001+np.abs(rPeakIntervalMean)),
                         np.log(.001+np.abs(qrsIntervalMean)),
                         np.log(.001+np.abs(pPeaksMean)),
                         np.log(.001+np.abs(tPeaksMean)),
                         np.log(.001+np.abs(qPeaksMean)),
                         np.log(.001+np.abs(sPeaksMean)),
                         np.log(.001+np.abs(tOffsetMean)),
                         np.log(.001+np.abs(pOnsetMean)),
                         np.log(.001+rPeakIntervalStd),
                         np.log(.001+qrsIntervalStd),
                         np.log(.001+pPeaksStd),
                         np.log(.001+tPeaksStd),
                         np.log(.001+rPeaksStd),
                         np.log(.001+qPeaksStd),
                         np.log(.001+sPeaksStd),
                         np.log(.001+tOffsetStd),
                         pOnsetStd])

    return np.array(features)

In [325]:
#build feature vector
pat0 = X_train.loc[0].dropna().to_numpy(dtype='float32')
feature_vec = feature_extraction(pat0)

for i in tqdm.trange(1, X_train.shape[0]):
    pat_i = X_train.loc[i].dropna().to_numpy(dtype='float32')
    feature_i = feature_extraction(pat_i)
    feature_vec = np.concatenate((feature_vec, feature_i))

  0%|▏                                                            | 14/5116 [00:03<18:20,  4.64it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  0%|▎                                                            | 24/5116 [00:05<19:30,  4.35it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  1%|▍                                    

  6%|███▊                                                        | 321/5116 [01:10<20:42,  3.86it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  6%|███▊                                                        | 323/5116 [01:11<21:14,  3.76it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
  7%|███▉                                 

 12%|███████▎                                                    | 626/5116 [02:23<15:48,  4.73it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 13%|████████                                                    | 688/5116 [02:38<11:37,  6.34it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 14%|████████▌                            

 20%|███████████▉                                               | 1038/5116 [04:03<16:36,  4.09it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 21%|████████████▍                                              | 1076/5116 [04:12<18:02,  3.73it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 21%|████████████▌                        

 29%|█████████████████                                          | 1477/5116 [05:52<13:55,  4.36it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 29%|█████████████████▏                                         | 1495/5116 [05:57<12:54,  4.68it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 30%|█████████████████▍                   

 36%|█████████████████████▏                                     | 1837/5116 [07:20<12:04,  4.52it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 36%|█████████████████████▍                                     | 1859/5116 [07:26<13:10,  4.12it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 37%|█████████████████████▋               

 42%|████████████████████████▋                                  | 2137/5116 [08:37<11:58,  4.14it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 42%|████████████████████████▉                                  | 2159/5116 [08:42<10:01,  4.92it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 43%|█████████████████████████            

 50%|█████████████████████████████▍                             | 2554/5116 [10:16<10:15,  4.17it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 50%|█████████████████████████████▋                             | 2571/5116 [10:20<09:09,  4.63it/s]/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/paul/miniforge3/envs/tensorflow/lib/python3.9/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 51%|██████████████████████████████       

ValueError: zero-size array to reduction operation fmax which has no identity

In [ ]:
feature_vec.shape

In [ ]:
df_features = pd.DataFrame(feature_vec)

In [ ]:
df_features

In [ ]:
df_features.to_csv('features_ata.csv')